In [ ]:
# 这会将你的Google Drive挂载到Colab虚拟机上。
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # 强制重新挂载Google Drive到指定路径

# 代办: 在你的Drive中输入保存了解压后的作业文件夹的路径，
# 例如 'cs231n/assignments/assignment3/'
FOLDERNAME = "cs231n/assignments/assignment3/"  # 作业文件夹在Google Drive中的路径
assert FOLDERNAME is not None, "[!] 请输入文件夹名称。"  # 确保已设置文件夹路径

# 现在我们已经挂载了你的Drive，这行代码确保
# Colab虚拟机的Python解释器能够从其中加载
# Python文件。
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))  # 将作业文件夹路径添加到Python的模块搜索路径中

In [ ]:
# 这会将COCO数据集下载到你的Drive中（如果尚未存在）
#（你应该已经从之前的笔记本中获取了这个数据集！）
# 如果没有的话，取消下面几行的注释。
# %cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/  # 切换到数据集存放目录
# !bash get_coco_captioning.sh  # 执行下载COCO字幕数据集的脚本
# %cd /content/drive/My\ Drive/$FOLDERNAME  # 切换回作业主目录

In [ ]:
# 一些有用的Python库
! pip install ftfy regex tqdm  # 安装ftfy（处理文本编码问题）、regex（正则表达式）、tqdm（进度条）
! pip install git+https://github.com/openai/CLIP.git  # 从GitHub安装OpenAI的CLIP库
! pip install decord  # 安装decord（高效的视频帧提取库）

# 最先进的预训练图像模型

在之前的练习中，你了解了[SimCLR](https://arxiv.org/abs/2002.05709)，以及对比自监督学习如何用于学习有意义的图像表征。在本笔记本中，我们将探讨另外两个较新的模型，它们也旨在学习高质量的视觉表征，并在各种下游任务中展现出了强大且稳健的性能。


首先，我们将研究[CLIP](https://github.com/openai/CLIP)模型。与SimCLR一样，CLIP使用对比学习目标，但它不是对比同一图像的两个增强视图，而是对比两种不同的模态：文本和图像。为了训练CLIP，OpenAI从互联网上收集了一个包含约4亿个图像-文本对的大型数据集，来源包括维基百科和图像替代文本等。由此产生的模型学习到了丰富的、高级的图像特征，并在许多视觉基准测试中实现了令人印象深刻的零样本性能。

接下来，我们将探讨[DINO](https://github.com/facebookresearch/dino)，这是一种用于视觉任务的自监督学习方法，它在自蒸馏框架中应用对比学习，并采用多裁剪增强策略。作者表明，DINO ViTs学习到的特征具有细粒度和丰富的语义，包含关于图像语义分割的明确信息。




# CLIP（对比语言-图像预训练）

如上所述，CLIP的训练目标融合了文本和图像，以对比学习原理为基础。回想SimCLR笔记本中的这句话：
>对比损失的目标是最大化最终向量 **$z_i = g(h_i)$** 和 **$z_j = g(h_j)$** 之间的一致性。

类似地，CLIP的训练目标是最大化两个向量之间的一致性。但由于这些向量来自不同的模态，CLIP使用了两个独立的编码器：基于Transformer的文本编码器和基于视觉Transformer（ViT）的图像编码器。需要注意的是，一些更小、更高效的CLIP版本使用ResNet作为图像编码器，而非ViT。

运行下方单元格，可视化CLIP的训练和推理流程。

在预训练阶段，每个批次包含多张图像及其对应的标题。每张图像由图像编码器（通常是视觉模型，如视觉Transformer（ViT）或卷积神经网络（ConvNet））独立处理，生成图像嵌入$I_n$。同样，每个标题由文本编码器独立处理，生成对应的文本嵌入$T_n$。接下来，我们计算所有图像-文本组合的 pairwise 相似度，即每张图像与每个标题进行比较，反之亦然。训练目标是最大化所得相似度矩阵对角线上的相似度分数——也就是匹配的图像-标题对$(I_n, T_n)$的分数。通过反向传播，模型学会为真实匹配对分配比错配对更高的相似度分数。

通过这种设置，CLIP有效地学习到了在共享潜在空间中表示图像和文本的能力。在这个空间中，语义概念以模态无关的方式编码，使得视觉和文本输入之间的跨模态比较成为可能。


In [ ]:
from IPython.display import Image as ColabImage  # 从IPython.display导入Image类并命名为ColabImage，用于在Colab中显示图像
# 显示位于Google Drive中指定路径的CLIP.png图像
ColabImage(f'/content/drive/My Drive/{FOLDERNAME}/CLIP.png')

**内联问题 1** ：

为什么CLIP的学习依赖于批量大小？如果批量大小固定，我们可以采用什么策略来学习丰富的图像特征？

$\color{blue}{\textit 你的答案：}$




# 加载COCO数据集

我们将使用你训练RNN字幕生成模型时用过的同一个字幕数据集，但这次我们不生成字幕，而是看看能否将每张图像与正确的字幕匹配起来。

In [ ]:
%load_ext autoreload  # 加载autoreload扩展，用于自动重新加载修改过的模块
%autoreload 2  # 设置autoreload模式为2，即自动重新加载所有导入的模块

import time, os, json  # 导入时间、操作系统、JSON处理相关模块
import numpy as np  # 导入numpy库，用于数值计算
import matplotlib.pyplot as plt  # 导入matplotlib，用于绘图
import torch  # 导入PyTorch库
import clip  # 导入CLIP库
import torch
from tqdm.auto import tqdm  # 从tqdm导入进度条工具

from PIL import Image  # 从PIL库导入Image，用于图像处理
from cs231n.clip_dino import *  # 从自定义模块导入所有内容

def rel_error(x, y):
    """计算相对误差。"""
    return np.max(np.abs(x - y) / (np.maximum(1e-10, np.abs(x) + np.abs(y))))  # 相对误差计算公式，避免除零


In [ ]:
# 从cs231n.coco_utils模块导入用于处理COCO数据集的工具函数
# load_coco_data：加载COCO数据集
# sample_coco_minibatch：从COCO数据集中采样小批量数据
# decode_captions：将字幕的索引序列解码为实际文本
from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions

# 从cs231n.image_utils模块导入根据URL获取图像的函数
from cs231n.image_utils import image_from_url

In [ ]:
# 将COCO数据从磁盘加载到字典中。
# 这和你在RNN字幕生成笔记本中使用的是同一个数据集 :)
data = load_coco_data(pca_features=True)  # 加载COCO数据，启用PCA特征

# 打印数据字典中的所有键和对应的值信息
for k, v in data.items():
    if type(v) == np.ndarray:  # 如果值是numpy数组，打印形状和数据类型
        print(k, type(v), v.shape, v.dtype)
    else:  # 否则打印长度（适用于列表等可迭代对象）
        print(k, type(v), len(v))

In [ ]:
# 我们只使用从COCO加载的字幕，因此需要对它们进行解码并去除特殊标记。
decoded_captions = []  # 用于存储解码后的字幕列表
for caption in data['val_captions']:  # 遍历验证集的字幕
    # 解码字幕（将索引转换为文字），然后去除<START>、<END>、<UNK>等特殊标记，并去除首尾空格
    caption = decode_captions(caption, data['idx_to_word'])\
        .replace('<START>', '')\
        .replace('<END>', '')\
        .replace('<UNK>', '')\
        .strip()
    decoded_captions.append(caption)  # 将处理后的字幕添加到列表中

In [ ]:
# 取10个示例
mask = np.array([135428, 122586, 122814, 133173, 176639, 163828,  98169,   6931,
        19488, 175760])  # 用于选取示例的掩码数组
first_captions = [decoded_captions[elem] for elem in mask]  # 根据掩码选取对应的解码字幕

img_idxs = data['val_image_idxs'][mask]  # 字幕所对应的图像索引
# 根据图像索引获取对应的URL，再通过URL获取图像
first_images = [image_from_url(data['val_urls'][j]) for j in img_idxs]

In [ ]:
# 遍历字幕和图像对，逐一显示
for i, (caption, image) in enumerate(zip(first_captions, first_images)):
    plt.imshow(image)  # 显示图像
    plt.axis('off')   # 关闭坐标轴显示
    caption_str = caption  # 获取字幕文本
    plt.title(caption_str)  # 将字幕设置为图像标题
    plt.show()  # 显示图像及其标题

# 运行CLIP模型

首先，我们将使用预训练的CLIP模型分别从文本和图像中提取特征。

In [ ]:
# 确定使用的设备：如果有可用的CUDA（GPU）就使用cuda，否则使用CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载预训练的CLIP模型和对应的预处理函数
# "ViT-B/32"表示使用的模型版本（ViT-Base，图像裁剪大小为32x32）
# 将模型加载到指定的设备（GPU或CPU）上
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# 你可以通过打印模型来查看模型的各层结构。
# CLIP的模型代码可在以下链接获取：https://github.com/openai/CLIP/tree/main/clip
# print(clip_model)  # 取消注释此行可打印模型结构

In [ ]:
# 首先，我们将字幕编码到共享嵌入空间中的向量。
# 由于我们使用Transformer作为文本编码器，需要先对文本进行分词。
text_tokens = clip.tokenize(first_captions).to(device)  # 对字幕进行分词处理并移至指定设备
with torch.no_grad():  # 禁用梯度计算，节省内存并加速计算
    text_features = clip_model.encode_text(text_tokens)  # 使用CLIP模型对文本进行编码，得到文本特征

# 完整性检查，打印特征形状
print(text_features.shape)

In [ ]:
# 然后，我们将图像编码到同一个嵌入空间中。
processed_images = [
    clip_preprocess(Image.fromarray(img)).unsqueeze(0)
    for img in first_images
]  # 对每张图像进行预处理（符合CLIP模型要求），并增加一个批次维度
images_tensor = torch.cat(processed_images, dim=0).to(device)  # 将所有图像张量拼接成一个批次，并移至指定设备

with torch.no_grad():  # 禁用梯度计算，节省内存并加速计算
    image_features = clip_model.encode_image(images_tensor)  # 使用CLIP模型对图像进行编码，得到图像特征

# 完整性检查，打印特征形状
print(image_features.shape)

打开 `cs231n/clip_dino.py` 文件，实现 `get_similarity_no_loop` 函数，用于计算文本特征和图像特征之间的相似度分数。在下方测试你的实现，你应该会看到相对误差小于 1e-5。

In [ ]:
# 从cs231n.clip_dino模块导入get_similarity_no_loop函数
from cs231n.clip_dino import get_similarity_no_loop

# 设置随机种子，确保结果可复现
torch.manual_seed(231)
np.random.seed(231)

# 定义测试用的维度参数：M个图像特征，N个文本特征，每个特征维度为D
M, N, D = 5, 6, 10

# 生成随机的测试文本特征和图像特征
test_text_features = torch.randn(N, D)  # 形状为[N, D]的文本特征
test_image_features = torch.randn(M, D)  # 形状为[M, D]的图像特征

# 使用实现的函数计算相似度分数
out = get_similarity_no_loop(test_text_features, test_image_features)

# 预期的输出结果（用于验证实现是否正确）
expected_out = np.array([
    [ 0.1867811 , -0.23494351,  0.44155994, -0.18950461,  0.00100103],
    [ 0.17905031, -0.25469488, -0.64330417,  0.25097957, -0.09327742],
    [-0.4407011 , -0.4365381 ,  0.32857686, -0.3765278 ,  0.01049389],
    [ 0.24815483,  0.42157224, -0.08459304,  0.14132318, -0.26935193],
    [ 0.02309848, -0.01441101,  0.5469337 ,  0.6018773 ,  0.21581158],
    [ 0.41579214, -0.014449  , -0.7242257 ,  0.39348006,  0.0822239 ],
]).astype(np.float32)

# 计算并打印实际输出与预期输出之间的相对误差
print("relative error: ", rel_error(out.numpy(), expected_out))

In [ ]:
# 可视化这批图像与其字幕之间的相似度

# 计算文本特征和图像特征之间的相似度，并转换为numpy数组
similarities = get_similarity_no_loop(text_features, image_features).cpu().detach().numpy()

plt.figure(figsize=(20, 14))  # 创建一个图形，设置大小为20x14
plt.imshow(similarities, vmin=0.1, vmax=0.3)  # 显示相似度矩阵，设置颜色映射的最小值和最大值

# 设置y轴刻度为字幕文本，字体大小为18；x轴不显示刻度
plt.yticks(range(len(text_features)), first_captions, fontsize=18)
plt.xticks([])

# 在图像上方显示对应的图片
for i, image in enumerate(first_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")

# 在矩阵的每个单元格中显示对应的相似度值，保留两位小数
for x in range(similarities.shape[1]):
    for y in range(similarities.shape[0]):
        plt.text(x, y, f"{similarities[y, x]:.2f}", ha="center", va="center", size=12)

# 隐藏图形四周的边框
for side in ["left", "top", "right", "bottom"]:
    plt.gca().spines[side].set_visible(False)

# 设置x轴和y轴的显示范围
plt.xlim([-0.5, len(image_features) - 0.5])
plt.ylim([len(text_features) + 0.5, -2])

# 设置图形标题，字体大小为20
plt.title("文本特征与图像特征之间的余弦相似度", size=20)
plt.show()  # 显示图形

# 零样本分类器

你会发现上面匹配的图像-字幕对之间具有很高的相似度。我们可以利用这一特性来设计一个不需要任何标记数据的图像分类器（即零样本分类器）。每个类别都可以用合适的自然语言描述来表示，任何输入图像都会被分类到其描述与该图像在CLIP嵌入空间中相似度最高的类别中。

在 `cs231n/clip_dino.py` 中实现 `clip_zero_shot_classifier` 函数，并在下方进行测试。你应该能得到以下预测结果：

['一个人', '一种动物', '一种动物', '食物', '一个人', '一幅风景画', '其他', '其他', '其他', '一个人']

In [ ]:
# 从cs231n.clip_dino模块导入零样本分类器函数
from cs231n.clip_dino import clip_zero_shot_classifier

# 定义分类类别列表
classes = ["a person", "an animal", "food", "a landscape", "other"]

# 使用CLIP模型进行零样本分类
# 参数包括：CLIP模型、预处理函数、待分类图像、类别列表和计算设备
pred_classes = clip_zero_shot_classifier(
    clip_model, clip_preprocess, first_images, classes, device)

# 打印分类预测结果
print(pred_classes)

运行下面的单元格以可视化预测结果。如你所见，CLIP提供了一种简单直接的方法，能够在任何类别分类体系上实现不错的零样本分类。

CLIP是第一个在ImageNet分类任务上超越标准有监督训练的模型，且它没有使用任何ImageNet的图像或标签（原始的CLIP论文中有许多这样有趣的实验和分析）。


In [ ]:
# 可视化零样本预测结果
for i, (pred_class, image) in enumerate(zip(pred_classes, first_images)):
    plt.imshow(image)  # 显示图像
    plt.axis('off')    # 关闭坐标轴显示
    plt.title(pred_class)  # 将预测的类别设置为图像标题
    plt.show()  # 显示图像及其标题

# 使用CLIP进行图像检索

就像我们使用CLIP为每个图像检索匹配的类别名称一样，我们也可以用它从文本输入中检索匹配的图像（语义图像检索）。在`cs231n/clip_dino.py`中实现`CLIPImageRetriever`，并通过运行下面两个单元格进行测试。每个查询的预期前2名输出已在注释中给出。

In [ ]:
# 从cs231n.clip_dino模块导入CLIPImageRetriever类
from cs231n.clip_dino import CLIPImageRetriever

# 实例化CLIP图像检索器
# 参数包括：CLIP模型、预处理函数、待检索的图像集合以及计算设备
clip_retriever = CLIPImageRetriever(clip_model, clip_preprocess, first_images, device)

In [ ]:
query = "sports"  # 相关运动：网球、滑板
# query = "black and white"  # 相关图像：浴室、斑马
# 根据查询文本检索相关图像的索引
img_indices = clip_retriever.retrieve(query)

# 显示检索到的图像
for img_index in img_indices:
    plt.imshow(first_images[img_index])  # 显示对应索引的图像
    plt.axis('off')  # 关闭坐标轴显示
    plt.show()  # 展示图像

**内联问题2** ：

CLIP通过对比损失在共享的潜在空间中学习对齐图像和文本表征。你会如何将这个想法扩展到两种以上的模态？

$\color{blue}{\textit 你的答案：}$


# DINO（深度对比自监督学习）

如前所述，使用SimCLR和CLIP等传统对比学习方法训练的模型需要非常大的批量大小。这使得它们计算成本高昂，并且限制了其可访问性。后续的研究工作，如[BYOL](https://arxiv.org/abs/2006.07733)，提出了一种替代方法，通过使用学生-教师框架来避免对大量负样本的需求。这种方法表现出惊人的效果，后来被[DINO](https://arxiv.org/abs/2104.14294)所采用。

与SimCLR类似，DINO的训练目标是最大化来自同一图像不同视图的两个向量之间的一致性。然而，与SimCLR不同的是，DINO使用两个独立的编码器，且它们的训练方式不同。学生网络通过反向传播进行更新，以匹配教师网络的输出。教师网络不通过反向传播更新，而是使用学生权重的指数移动平均（EMA）来更新其权重。这意味着教师模型的进化更缓慢，并为学生提供了一个稳定的学习目标。

运行下面的单元格以可视化DINO的训练流程。

In [ ]:
# 从IPython.display模块导入Image类并将其重命名为ColabImage（避免与图像处理的Image类冲突）
from IPython.display import Image as ColabImage

# 显示指定路径下的DINO训练流程动画GIF
# 路径为：/content/drive/My Drive/{FOLDERNAME}/dino.gif（其中{FOLDERNAME}是文件夹名称占位符）
ColabImage(f'/content/drive/My Drive/{FOLDERNAME}/dino.gif')

In [ ]:
# 首先，我们移除当前占用内存的CLIP模型
del clip_model
# 如果你使用的是GPU运行时，请取消下面几行的注释
# torch.cuda.empty_cache()  # 清空CUDA缓存
# torch.cuda.ipc_collect()  # 收集CUDA进程间通信的未使用内存

In [ ]:
# 加载最小的DINO模型：ViT-S/8。其中ViT-S（小尺寸视觉Transformer）约有2200万参数，
# 处理8x8大小的图像块。
dino_model = torch.hub.load('facebookresearch/dino:main', 'dino_vits8')  # 从PyTorch Hub加载预训练的DINO模型
dino_model.eval().to(device)  # 设置模型为评估模式，并移至指定设备（GPU或CPU）

In [ ]:
# 我们将要处理的图像
sample_image = Image.fromarray(first_images[0]).convert("RGB")  # 从数组创建图像对象，并转换为RGB格式
sample_image  # 在输出中显示该图像

# DINO注意力图

由于加载的DINO检查点基于ViT架构，我们可以可视化每个注意力头所关注的区域。下面的代码会生成热力图，展示在最后一层的各个注意力头中，[CLS]标记关注原始图像的哪些图像块。尽管这个模型是使用自监督目标训练的，没有任何明确的指令去识别图像中的“结构”，但依然……

你注意到什么模式了吗？

In [ ]:
# 预处理
from torchvision import transforms as T
# 定义预处理流程：调整大小、转换为张量、标准化
transform = T.Compose([
    T.Resize((480, 480)),  # 将图像调整为480x480大小
    T.ToTensor(),  # 转换为张量
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # 使用ImageNet的均值和标准差进行标准化
])
img_tensor = transform(sample_image)  # 对样本图像应用预处理
w, h = img_tensor.shape[1:]  # 获取图像的宽度和高度
img_tensor = img_tensor[None].to(device)  # 增加批次维度，并移至指定设备

# 提取注意力权重
with torch.no_grad():  # 禁用梯度计算
    # 获取最后一层的自注意力，取第0个样本，所有注意力头，[CLS]标记对其他所有标记的注意力
    attn = dino_model.get_last_selfattention(img_tensor)[0, :, 0, 1:]
nh, tokens = attn.shape  # nh是注意力头数量，tokens是标记数量
w_feat, h_feat = w // 8, h // 8  # 特征图的宽度和高度（由于使用8x8的patch，所以除以8）
attn = attn.reshape(nh, w_feat, h_feat)  # 将注意力权重重塑为(注意力头数量, 特征图宽度, 特征图高度)
# 将注意力图上采样到原始图像大小（缩放因子为8）
attn = torch.nn.functional.interpolate(attn.unsqueeze(0), scale_factor=8, mode="nearest")[0].cpu().numpy()

# 绘制注意力头的热力图
fig, axes = plt.subplots(1, nh, figsize=(3 * nh, 3))  # 创建1行nh列的子图
for i in range(nh):
    ax = axes[i] if nh > 1 else axes  # 处理只有一个注意力头的情况
    ax.imshow(attn[i], cmap='inferno')  # 显示注意力热力图，使用inferno颜色映射
    ax.axis('off')  # 关闭坐标轴
plt.show()  # 显示图像

In [ ]:
# 提取图像块标记特征并丢弃[CLS]标记
with torch.no_grad():  # 禁用梯度计算，节省内存并加快计算
    # 获取中间层特征，n=1表示取最后一个中间层
    # 返回结果形状为(1, 1+N, D)，其中1是[CLS]标记，N是图像块数量，D是特征维度
    all_tokens = dino_model.get_intermediate_layers(img_tensor, n=1)[0]
    # 截取从索引1开始的部分，即丢弃[CLS]标记，得到图像块标记特征
    # 形状为(N, D)，N是图像块数量，D是特征维度
    patch_tokens = all_tokens[:, 1:, :]

# 打印各张量的形状，用于验证
print(img_tensor.shape)    # 输入图像张量的形状
print(all_tokens.shape)    # 包含[CLS]标记的所有标记特征的形状
print(patch_tokens.shape)  # 仅包含图像块标记特征的形状

**内联问题3**：

我们是如何得到上面打印出的张量形状的？请解释你的答案。


$\color{blue}{\textit 你的答案：}$






# DINO特征

为了理解模型在每个图像块中编码了什么信息，我们可以可视化每个图像块标记的内容。由于这些嵌入是高维的，难以直接解读，我们将使用PCA（主成分分析）来确定特征空间中方差最大的方向。

在下一个单元格中，我们将可视化特征空间中三个主要的方差方向。这将揭示图像块嵌入所捕捉到的主要结构。

In [ ]:
# 从scikit-learn库导入PCA（主成分分析）工具
from sklearn.decomposition import PCA

# 设置随机种子，保证结果可复现
np.random.seed(231)

# 进行PCA降维
pca = PCA(n_components=3)  # 初始化PCA，设置降维到3个主成分
# 对图像块标记特征（已转移到CPU并转为numpy数组）进行PCA拟合和转换
patch_pca = pca.fit_transform(patch_tokens.cpu().numpy()[0])

# 将PCA得到的主成分标准化到[0, 1]区间，以便作为RGB颜色显示
patch_rgb = (patch_pca - patch_pca.min(0)) / (patch_pca.max(0) - patch_pca.min(0))

# 将标准化后的PCA结果重塑为60x60的图像网格形状（每个网格对应一个图像块），通道数为3（对应RGB）
patch_rgb_img = patch_rgb.reshape(60, 60, 3)

# 显示图像
plt.figure(figsize=(6, 6))  # 创建6x6英寸的图像窗口
plt.imshow(patch_rgb_img)   # 显示PCA降维后转换为RGB的图像块特征
plt.axis('off')             # 关闭坐标轴显示
plt.title("Patch Embeddings (PCA → RGB)")  # 设置标题：图像块嵌入（PCA→RGB）
plt.show()                  # 展示图像

**内联问题4** ：

在上面的可视化结果中，你看到了什么样的结构？当一个区域始终呈现特定颜色时，这意味着什么？当两个区域呈现截然不同的颜色时，又意味着什么？请记住，PCA揭示了所有图像块在特征空间中方差最大的方向。一个图像块的颜色反映了其独特的特征内容。


$\color{blue}{\textit 你的答案：}$




# 基于DINO特征的简单分割模型

在上一节中，我们看到DINO特征可以提供出人意料的良好分割线索。现在，让我们通过在[DAVIS数据集](https://davischallenge.org)上训练一个简单的分割模型来验证这个想法。DAVIS数据集（密集标注视频分割数据集）是为视频对象分割任务创建的，它提供了视频中对象的逐帧、像素级标注。在这个实验中，我们将仅使用一个视频的单帧标注来训练模型，然后观察它在该视频的其余帧上表现如何。

我们的模型会特意设计得非常精简：我们将提取每个图像块的DINO特征，然后仅使用那个带标注帧的图像块来训练一个轻量级的图像块分类器。通常情况下，你会在整个数据集上训练，然后在包含不同视频的独立验证集上评估。但在这里，我们将测试DINO特征的零样本（one-shot）能力。



In [ ]:
# 从cs231n.clip_dino模块导入DavisDataset类
from cs231n.clip_dino import DavisDataset

# 用于处理DAVIS数据集的辅助类
# 首次运行此单元格时，下载数据集可能需要约5分钟
davis_ds = DavisDataset()  # 实例化DAVIS数据集对象

# 获取特定的测试视频。提交时请勿修改此部分
frames, masks = davis_ds.get_sample(7)  # 获取样本编号为7的视频帧和对应的掩码
num_classes = masks.max() + 1  # 计算类别数量（掩码中最大值加1，因为掩码从0开始）

# 打印视频帧、掩码的形状以及类别数量
print(frames.shape, masks.shape, num_classes)

In [ ]:
# 获取中间某一帧的DINO图像块特征及对应的类别标签
train_fi = 40  # 训练所用帧的索引（第40帧）
# 处理第40帧的图像，提取DINO特征，取结果的第一个元素（因为只处理了一帧）
X_train = davis_ds.process_frames(frames[train_fi:train_fi+1], dino_model, device)[0]
# 处理第40帧对应的掩码，获取类别标签，取结果的第一个元素
Y_train = davis_ds.process_masks(masks[train_fi:train_fi+1], device)[0]

# 打印训练特征和标签的形状
print(X_train.shape, Y_train.shape)

在`cs231n/clip_dino.py`中完成`DINOSegmentation`类的实现，并通过运行下面两个单元格进行测试。在第一个测试帧上，你的平均交并比（mean IoU）应大于0.45，在最后一个测试帧上应大于0.50。为了防止在训练图像块特征上发生过拟合，可以考虑设计一个非常轻量的模型（例如，一个线性层或两层的多层感知机（MLP）），并应用适当的权重衰减。

你可以使用GPU运行时来加快训练和评估速度。如果更改了运行时类型，请确保重新运行整个笔记本。

In [ ]:
# 从cs231n.clip_dino模块导入DINOSegmentation类和compute_iou函数
from cs231n.clip_dino import DINOSegmentation, compute_iou

# 设置随机种子，保证实验结果可复现
torch.manual_seed(231)
np.random.seed(231)

# 实例化DINO分割模型，传入计算设备和类别数量
dino_segmentation = DINOSegmentation(device, num_classes)
# 训练模型，使用训练特征X_train和标签Y_train，迭代500次
dino_segmentation.train(X_train, Y_train, num_iters=500)


# 在首帧、中间帧和末帧上进行测试
ious = []  # 用于存储各测试帧的IoU（交并比）
test_fis = [0, train_fi, 98]  # 测试帧的索引：首帧、训练所用帧、末帧
gt_viz = []  # 用于存储带真实掩码的可视化图像
pred_viz = []  # 用于存储带预测掩码的可视化图像

for fi in test_fis:
    # 处理当前测试帧，提取DINO特征，取结果的第一个元素（因仅处理一帧）
    X_test = davis_ds.process_frames(frames[fi:fi+1], dino_model, device)[0]
    # 获取当前测试帧对应的真实掩码，取结果的第一个元素
    Y_test = davis_ds.process_masks(masks[fi:fi+1], device)[0]
    # 对测试特征进行推理，得到预测掩码
    Y_pred = dino_segmentation.inference(X_test)
    # 计算预测掩码与真实掩码的IoU，并添加到列表中
    iou = compute_iou(Y_pred, Y_test, num_classes)
    ious.append(iou)

    # 生成带真实掩码的叠加图像并添加到列表
    gt_viz.append(davis_ds.mask_frame_overlay(Y_test, frames[fi]))
    # 生成带预测掩码的叠加图像并添加到列表
    pred_viz.append(davis_ds.mask_frame_overlay(Y_pred, frames[fi]))

# 将多个测试帧的真实掩码可视化结果横向拼接
gt_viz = np.concatenate(gt_viz, 1)
# 将多个测试帧的预测掩码可视化结果横向拼接
pred_viz = np.concatenate(pred_viz, 1)

In [ ]:
# 打印第一个测试帧的平均交并比（保留三位小数）
# 要求结果应大于0.45
print(f"第一个测试帧的平均IoU：{ious[0]:.3f}")
# 打印最后一个测试帧的平均交并比（保留三位小数）
# 要求结果应大于0.50
print(f"最后一个测试帧的平均IoU：{ious[2]:.3f}")

现在让我们将结果可视化。运行下面两个单元格，以显示首帧、中间帧和末帧的真实分割掩码与预测分割掩码。注意，中间帧属于训练集，而其他帧是未见过的。

In [ ]:
# 将真实掩码可视化结果（numpy数组）转换为图像对象并显示
Image.fromarray(gt_viz)

In [ ]:
# 将预测掩码可视化结果（numpy数组）转换为图像对象并显示
Image.fromarray(pred_viz)

现在运行下面三个单元格来评估并可视化整个视频。你的平均交并比（mean IoU）应大于0.45。保存在谷歌云端硬盘中的可视化视频可能需要一些时间处理，但你可以将其下载到电脑上本地查看。



In [ ]:
# 在所有帧上运行模型
ious = []  # 存储所有帧的交并比
gt_viz = []  # 存储所有帧的真实掩码可视化结果
pred_viz = []  # 存储所有帧的预测掩码可视化结果

for fi in range(len(frames)):
    # 每处理20帧打印一次进度
    if fi % 20 == 0:
        print(f"{fi} / {len(frames)}")  # 当前处理的帧索引 / 总帧数
    
    # 提取当前帧的DINO特征
    X_test = davis_ds.process_frames(frames[fi:fi+1], dino_model, device)[0]
    # 获取当前帧的真实掩码
    Y_test = davis_ds.process_masks(masks[fi:fi+1], device)[0]
    # 对当前帧进行分割预测
    Y_pred = dino_segmentation.inference(X_test)
    # 计算并存储当前帧的交并比
    iou = compute_iou(Y_pred, Y_test, num_classes)
    ious.append(iou)
    
    # 生成并存储带真实掩码的叠加图像
    gt_viz.append(davis_ds.mask_frame_overlay(Y_test, frames[fi]))
    # 生成并存储带预测掩码的叠加图像
    pred_viz.append(davis_ds.mask_frame_overlay(Y_pred, frames[fi]))

# 将列表转换为数组，形状为：T(帧数) x H(高度) x W(宽度) x 3(通道)
gt_viz = np.stack(gt_viz)
pred_viz = np.stack(pred_viz)

# 将真实掩码可视化和预测掩码可视化在宽度方向拼接，形状为：T x H x 2W x 3
final_viz = np.concatenate([gt_viz, pred_viz], -2)

In [ ]:
# 计算所有帧的平均交并比（IoU）并打印（保留三位小数）
# 要求结果应大于0.55
print(f"所有帧的平均IoU：{sum(ious) / len(ious):.3f}")


In [ ]:
def write_video_from_array(array, output_path, fps=12):
    """
    将图像数组写入视频文件
    
    参数:
    array: 输入的图像数组，形状应为[T(帧数), H(高度), W(宽度), 3(通道数)]
    output_path: 视频保存路径
    fps: 视频帧率，默认12帧/秒
    """
    T, H, W, _ = array.shape  # 解析数组维度：帧数、高度、宽度、通道数
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 设置视频编码格式为mp4v
    # 创建视频写入器，指定输出路径、编码格式、帧率和分辨率
    out = cv2.VideoWriter(output_path, fourcc, fps, (W, H))
    for i in range(T):
        frame = array[i]  # 提取当前帧图像
        out.write(frame)  # 写入视频帧
    out.release()  # 释放视频写入器资源
    print(f"Video saved to {output_path}")  # 打印保存成功的提示信息


# 在谷歌云端硬盘中处理可能需要一段时间，但你可以直接下载到电脑观看
write_video_from_array(final_viz, f"/content/drive/My Drive/{FOLDERNAME}/dino_res.mp4")

**内联问题5** ：

如果在CLIP ViT的图像块特征上训练分割模型，你认为其性能会比DINO更好还是更差？为什么？


$\color{blue}{\textit 你的答案：}$


